In [28]:
import requests
import osmnx as ox
import json
import geopandas as gpd

APIKEY = "my_key"
ZONE = "Paris, France"
ROUNDABOUTS = None

In [29]:
from shapely.geometry import mapping

ROUNDABOUTS = ox.features_from_place(ZONE, tags={'junction': 'roundabout'})
ROUNDABOUTS['center'] = ROUNDABOUTS.centroid

ROUNDABOUTS = ROUNDABOUTS.reset_index(drop=True)
ROUNDABOUTS['id'] = (ROUNDABOUTS.index + 1).astype(int)

ROUNDABOUTS = ROUNDABOUTS[['id', 'center', 'geometry']]

features = []
for _, row in ROUNDABOUTS.iterrows():
    features.append({
        "type": "Feature",
        "id": int(row['id']),
        "properties": {
            "id": int(row['id']),
            "center": [row['center'].x, row['center'].y]
        },
        "geometry": mapping(row['geometry'])
    })

geojson = {"type": "FeatureCollection", "features": features}

with open("roundabouts.json", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"Nombre de ronds-points trouvés : {len(ROUNDABOUTS)}")


Nombre de ronds-points trouvés : 7


C:\Users\mmini\AppData\Local\Temp\ipykernel_27248\3360671764.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ROUNDABOUTS['center'] = ROUNDABOUTS.centroid


In [30]:
def recup_image_google_street_view(lat, lon, api_key=APIKEY, heading=0, fov=90, pitch=0, size="600x400"):
    url = f"https://maps.googleapis.com/maps/api/streetview?size={size}&location={lat},{lon}&fov={fov}&heading={heading}&pitch={pitch}&key={api_key}"
    response = requests.get(url)